# Database Admin 101 - Lab

## Introduction 

In this lab, you'll go through the process of designing and creating a database. From there, you'll begin to populate this table with mock data provided to you.

## Objectives

You will be able to:

* Use knowledge of the structure of databases to create a database and populate it

## The Scenario

You are looking to design a database for a school that will house various information from student grades to contact information, class roster lists and attendance. First, think of how you would design such a database. What tables would you include? What columns would each table have? What would be the primary means to join said tables?

## Creating the Database

Now that you've put a little thought into how you might design your database, it's time to go ahead and create it! Start by import the necessary packages. Then, create a database called **school.sqlite**.

In [1]:
# Import necessary packages
import sqlite3
import pandas as pd

In [2]:
# Create the database school.sqlite 
conn = sqlite3.connect("school.sqlite")
curr = conn.cursor()


## Create a Table for Contact Information

Create a table called contactInfo to house contact information for both students and staff. Be sure to include columns for first name, last name, role (student/staff), telephone number, street, city, state, and zipcode. Be sure to also create a primary key for the table. 

In [3]:
# Your code here
curr.execute("""
CREATE TABLE IF NOT EXISTS contactInfo(
            firstName TEXT,
            lastName TEXT,
            role TEXT,
            telephone INTERGER PRIMARY KEY,
            street TEXT ,
            city TEXT,
            state TEXT,
            zipcode TEXT )
""")

In [4]:
q = """
SELECT *
FROM contactInfo
;
"""
pd.read_sql(q,conn).info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   firstName  0 non-null      object
 1   lastName   0 non-null      object
 2   role       0 non-null      object
 3   telephone  0 non-null      object
 4   street     0 non-null      object
 5   city       0 non-null      object
 6   state      0 non-null      object
 7   zipcode    0 non-null      object
dtypes: object(8)
memory usage: 0.0+ bytes


## Populate the Table

Below, code is provided for you in order to load a list of dictionaries. Briefly examine the list. Each dictionary in the list will serve as an entry for your contact info table. Once you've briefly investigated the structure of this data, write a for loop to iterate through the list and create an entry in your table for each person's contact info.

In [5]:
# Load the list of dictionaries; just run this cell
import pickle

with open('contact_list.pickle', 'rb') as f:
    contacts = pickle.load(f)

In [104]:
# Iterate over the contact list and populate the contactInfo table here

for i in range(len(contacts)):
    Key_name=contacts[i].values()       
    curr.execute("""
    INSERT INTO contactInfo (firstName ,lastName ,role,telephone, street,city, state, zipcode)
                VALUES (?,?,?,?,?,?,?,?)
                
    """,tuple([x for x in Key_name]))

**Query the Table to Ensure it is populated**

In [106]:
# Your code here 
q="""SELECT * FROM contactInfo;"""
pd.read_sql(q,conn)

,firstName,lastName,role,telephone,street,city,state,zipcode
0,Christine,Holden,staff,2035687697,1672 Whitman Court,Stamford,CT,06995
1,Christopher,Warren,student,2175150957,1935 University Hill Road,Champaign,IL,61938
2,Linda,Jacobson,staff,4049446441,479 Musgrave Street,Atlanta,GA,30303
3,Andrew,Stepp,student,7866419252,2981 Lamberts Branch Road,Hialeah,Fl,33012
4,Jane,Evans,student,3259909290,1461 Briarhill Lane,Abilene,TX,79602
5,Jane,Evans,student,3259909290,1461 Briarhill Lane,Abilene,TX,79602
6,Mary,Raines,student,9075772295,3975 Jerry Toth Drive,Ninilchik,AK,99639
7,Ed,Lyman,student,5179695576,3478 Be Sreet,Lansing,MI,48933


## Commit Your Changes to the Database

Persist your changes by committing them to the database.

In [107]:
# Your code here
conn.commit()

## Create a Table for Student Grades

Create a new table in the database called "grades". In the table, include the following fields: userId, courseId, grade.

** This problem is a bit more tricky and will require a dual key. (A nuance you have yet to see.)
Here's how to do that:

```SQL
CREATE TABLE table_name(
   column_1 INTEGER NOT NULL,
   column_2 INTEGER NOT NULL,
   ...
   PRIMARY KEY(column_1,column_2,...)
);
```

In [109]:
# Create the grades table

curr.execute("""
CREATE TABLE grades(
             userId INTERGER NOT NULL,
             courseId INTERGER NOT NULL,
             grade,
             PRIMARY KEY (userId,courseId));
""")

## Remove Duplicate Entries

An analyst just realized that there is a duplicate entry in the contactInfo table! Find and remove it.

In [112]:
# Find the duplicate entry
q="""
SELECT telephone ,firstName, lastName, COUNT(*) AS count
FROM contactInfo
GROUP BY telephone, firstName
HAVING count > 1
;
"""
pd.read_sql(q,conn)

,telephone,firstName,lastName,count
0,3259909290,Jane,Evans,2


In [113]:
# Delete the duplicate entry
curr.execute("""
DELETE FROM contactInfo
WHERE ROWID NOT IN (
             SELECT MIN(ROWID)
             FROM contactInfo
             GROUP BY telephone ,firstName);
""")
conn.commit()


,firstName,lastName,role,telephone,street,city,state,zipcode
0,Christine,Holden,staff,2035687697,1672 Whitman Court,Stamford,CT,06995
1,Christopher,Warren,student,2175150957,1935 University Hill Road,Champaign,IL,61938
2,Linda,Jacobson,staff,4049446441,479 Musgrave Street,Atlanta,GA,30303
3,Andrew,Stepp,student,7866419252,2981 Lamberts Branch Road,Hialeah,Fl,33012
4,Jane,Evans,student,3259909290,1461 Briarhill Lane,Abilene,TX,79602
5,Mary,Raines,student,9075772295,3975 Jerry Toth Drive,Ninilchik,AK,99639
6,Ed,Lyman,student,5179695576,3478 Be Sreet,Lansing,MI,48933


In [114]:
# Check that the duplicate entry was removed
pd.read_sql("""SELECT * FROM contactInfo;""",conn)

,firstName,lastName,role,telephone,street,city,state,zipcode
0,Christine,Holden,staff,2035687697,1672 Whitman Court,Stamford,CT,06995
1,Christopher,Warren,student,2175150957,1935 University Hill Road,Champaign,IL,61938
2,Linda,Jacobson,staff,4049446441,479 Musgrave Street,Atlanta,GA,30303
3,Andrew,Stepp,student,7866419252,2981 Lamberts Branch Road,Hialeah,Fl,33012
4,Jane,Evans,student,3259909290,1461 Briarhill Lane,Abilene,TX,79602
5,Mary,Raines,student,9075772295,3975 Jerry Toth Drive,Ninilchik,AK,99639
6,Ed,Lyman,student,5179695576,3478 Be Sreet,Lansing,MI,48933


## Updating an Address

Ed Lyman just moved to `2910 Simpson Avenue York, PA 17403`. Update his address accordingly.

In [116]:
# Update Ed's address
curr.execute("""
UPDATE contactInfo
             SET street = '2910 Simpson Avenue York'
             WHERE street ='3478 Be Sreet';
""")
curr.execute("""
UPDATE contactInfo
             SET city = 'New York'
             WHERE city ='Lansing';
""")
curr.execute("""
UPDATE contactInfo
             SET state = 'PA'
             WHERE state ='MI';
""")


In [117]:
# Query the database to ensure the change was made
q="""SELECT * FROM contactInfo;"""
pd.read_sql(q,conn)

,firstName,lastName,role,telephone,street,city,state,zipcode
0,Christine,Holden,staff,2035687697,1672 Whitman Court,Stamford,CT,06995
1,Christopher,Warren,student,2175150957,1935 University Hill Road,Champaign,IL,61938
2,Linda,Jacobson,staff,4049446441,479 Musgrave Street,Atlanta,GA,30303
3,Andrew,Stepp,student,7866419252,2981 Lamberts Branch Road,Hialeah,Fl,33012
4,Jane,Evans,student,3259909290,1461 Briarhill Lane,Abilene,TX,79602
5,Mary,Raines,student,9075772295,3975 Jerry Toth Drive,Ninilchik,AK,99639
6,Ed,Lyman,student,5179695576,2910 Simpson Avenue York,New York,PA,48933


## Commit Your Changes to the Database

Once again, persist your changes by committing them to the database.

In [118]:
# Your code here
conn.commit()
conn.close()

## Summary

While there's certainly more to do with setting up and managing this database, you got a taste for creating, populating, and maintaining databases! Feel free to continue fleshing out this exercise for more practice. 